In [ ]:
pip install --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
reviews_path = "/content/drive/MyDrive/bdsa_data/Musical_Instruments.json" #change to the Drive path where the review data is stored

In [ ]:
products_considered = ["B00I11N2VO","B00I13N9NI","B00TY2X188","B000L6GD04"] #a few example products from dataset

In [ ]:
import json

# Assuming the JSON file is named "your_file.json"
with open(reviews_path, "r") as file:
    reviews = {}

    # Iterate through each line in the JSON file
    for line in file:
        # Load each line as JSON
        try:
            row = json.loads(line)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON: {e}")
            continue  # Skip to the next line if there"s an error

        # Extract relevant information
        asin = row.get("asin")
        if asin not in products_considered:
          continue
        overall = row.get("overall")
        review_text = row.get("reviewText")
        review_time = row.get("reviewTime")

        # Create a dictionary for each review
        review_info = {
            "rating": overall,
            "reviewText": review_text,
            "reviewTime": review_time
        }

        # Check if the asin is already in the dictionary
        if asin in reviews:
            reviews[asin].append(review_info)
        else:
            reviews[asin] = [review_info]

In [ ]:
for product in products_considered:
    print("Number of reviews for", product, ":", len(reviews[product]))

Number of reviews for B00I11N2VO : 45
Number of reviews for B00I13N9NI : 163
Number of reviews for B00TY2X188 : 40
Number of reviews for B000L6GD04 : 608


In [ ]:
PROMPT_20 = "You are an Amazon AI shopping assistant. Your task is summarizing multiple amazon reviews for a given product under these headings \"Whats good about the product\" (Should cover only the positive thing about the product not about shipping or delivery or quickness of delivery only the product), \"whats bad about the product (specifics about what is bad about the product not shipping or delivery)\", \"Delivery issues (Any shipping or damage or delivery issues that came up)\". You have to make the review as information rich as possible and as useful for the customer. You also have to come up with a net rating out of 5, using existing user ratings from the field \“rating\”, and also justify why you gave that rating. You have to generate the summary for all the attached 20 or less reviews. Now let us see how will you get the input responses. The input responses will be json format a list of kind {\'rating\': 5.0,\'reviewText\': \'Better than i expect\',\'reviewTime\': \'12 13, 2016\'}. You have to summarize the entire list of reviews. Now lets see how you generate the summary.Your task is to give a Summarized 100-150 word review under the subheadings of \"Whats good about the product\" (Should cover only the positive thing about the product not about shipping or delivery or quickness of delivery only the product), \"whats bad about the product (specifics about what is bad about the product not shipping or delivery)\", \"Delivery issues (Any shipping or damage or delivery issues that came up)\" ,Make it a nice summary which easy for a customer to read, this should help the customer with his/her shopping experience. Make sure to only give summaries under the asked headings by using function calling. Also note that sometimes the input lenght can be as low as 1 review, so always use function calling and return a summary as instructed."
PROMPT_latest = "You are an Amazon AI shopping assistant, which summarizes the latest reviews of the product. Your task is summarizing the latest 20 multiple amazon reviews for a given product under its pros and cons. Do not use bullets or headings, just a 3-4 line review of what are the pros and cons of the product. You have to make the review as information rich as possible and as useful for the user. For such a prompt, you have to generate the summary for all the attached 20 reviews. Now let us see how will you get the input responses. The input responses will be json format a list of kind {\'rating\': 5.0,\'reviewText\': \'Better than i expect\',\'reviewTime\': \'12 13, 2016\'}. You have to summarize the entire list of reviews. Now lets see how you generate the summary.Your task is to give a Summarized 80 word review. Make it a nice summary which easy for a customer to read, this should help the customer with his/her shopping experience. "
PROMPT_COMBINE = "You are a summary combiner for Amazon AI shopping assistant. You will receive a stringified list of json summary objects. Your task is Combining multiple summaries generated together into a SINGLE summary. Each of the summaries will have the same json format. You have to combine all the summaries together into a summary under the same format, within 100-150 words. REMEMBER it is a review, you should not miss out on any key information mentioned about the product while combining the reviews. So do it in a way that your summary helps the user in shopping. Each summary will also have a net rating. So you also have to come up with a combined net rating for these summarie. Always use function calling while combining these summary and return responses in a deterministic json format as instructed"
PROMPT_RATING = "You are an Amazon AI shopping assistant. Your task is summarizing multiple amazon reviews for a given product, under 5 rating headings. You have to make the review as information rich as possible and as useful for the customer. You also have to come up with a net rating out of 5, using existing user ratings from the field \“rating\”, and also justify why you gave that rating. The input responses will be json format a list of kind {\'rating\': 5.0,\'reviewText\': \'Better than i expect\',\'reviewTime\': \'12 13, 2016\'}. You have to summarize the entire list of 20 reviews, in the current response, including the ones from the previous responses.Your task is to give a Summarised 80 word review under the subheadings of \"Rating : 1\" (Should only summaries reviews with \'rating\' as 1),\"Rating : 2\" (Should only summaries reviews with \'rating\' as 2), \"Rating : 3\" (Should only summaries reviews with \'rating\' as 3), \"Rating : 4\" (Should only summaries reviews with \'rating\' as 4), and \"Rating : 5\" (Should only summaries reviews with \'rating\' as 5). Make it a nice summary which easy for a customer to read, this should help the customer with his/her shopping experience. Also mention the support for each of the issues and positives, that is how many reviews support those claims."
PROMPT_COMBINE_RATING = "You are a summary combiner and for Amazon AI shopping assistant. Your task is Combining multiple summaries generated together into a single summary. Each of the summaries will have the same subheadings,\"Rating : 1\" ,\"Rating : 2\" , \"Rating : 3\" , \"Rating : 4\" , and \"Rating : 5\" . You have to combine all the summaries together into a summary under the same headings, within 100 words. REMEMBER it is a review, you should not miss out on any key information mentioned about the product while combining the reviews. So do it in a way that your summary helps the user in shopping. Take care that you incorporate all records of reviews and are correct mathematically. Make sure if some summary has a review for a specific rating that it is included"
PROMPT_TEXT_JSON_COMBINE = "You are given a summary of a product. You need to return this summary as a JSON with the fields: \"Whats good about the product\" (Should cover only the positive thing about the product not about shipping or delivery or quickness of delivery only the product), \"whats bad about the product (specifics about what is bad about the product not shipping or delivery)\", \"Delivery issues (Any shipping or damage or delivery issues that came up)\", \"Net Rating (A numeric rating out of 5 where 0 indicates bad product and 5 indicates amazing product)\", \"Justification (A justification for the net rating you provided)\". Follow function calling for the exact JSON format. Your response should at least have 100 words."
RECOMMEND_PROMPT = "You are a product recommender, the user will provide a description of what kind of a product they are looking for. you will also be given a json file of product information as text input. The json will have the product asin as key, and the values will have the meta information features as key and values, along with review as a key. You should recommend giving high weightage to the reviews, and also to the product information. You have to output the asin (key) of the product. You only recommend a single product."
GET_TAGS = "You are an amazon attribute generator. Given a  set of 30 product reviews, you need to extract the attributes of the product from the reviews. The attribute is a characteristic that the product possesses that can be described as good or bad. The attribute does not denote what the product is, or what variant of the product it is. I need a list of EXACTLY 6 attributes. No other filler text in your response"

In [ ]:
#summarization function call
functions = [
    {
      "name": "formatted_summary",
      "description": "Format the review summary in JSON for a product",
      "parameters": {
        "type":"object",
        "properties": {
            "Whats good about the product": {
                "type": "string",
                "description": "Should cover only the positive thing about the product not about shipping or delivery or quickness of delivery only the product. Within 50 words"
            },
            "Whats bad about the product": {
                "type": "string",
                "description": "Specifics about what is bad about the product not shipping or delivery. Within 50 words"
            },
            "Delivery issues": {
                "type": "string",
                "description": "Any shipping or damage or delivery issues that came up. Within 20 words"
            },
            "Net Rating": {
                "type": "number",
                "description": "Overall Rating of the Product out of 5. It is a float value"
            },
            "Justification": {
                "type": "string",
                "description": "Justification for the rating, within 20 words"
            }
        }
  }
    }
]

#attributes function call
functions_attribute = [
    {
      "name": "attributes",
      "description": "Extract attributes from reviews",
      "output": {
        "type":"array",
      "items": {
          "type": "string"
      }
  }
    }
]

#combine summaries function call
functions_combine = [
    {
      "name": "formatted_combined_summary",
      "description": "You are given a list of jsons representing the summary for a product. Each json contains the fields - 'Whats good about the product', 'Whats bad about the product', 'Delivery issues', 'Net Rating' and 'Justification'. Summarize and combine these jsons into a single json with the same 5 fields.",
      "parameters": {
        "type":"object",
        "properties": {
            "Whats good about the product": {
                "type": "string",
                "description": "should contain the combined summary of the input json field 'Whats good about the product' across all the given summaries, within 70 words"
            },
            "Whats bad about the product": {
                "type": "string",
                "description": "should contain the combined summary of the input json field 'Whats bad about the product' values across all the given summaries, within 70 words"
            },
            "Delivery issues": {
                "type": "string",
                "description": "should contain the combined summary of the input json field 'delivery issues' across all the given summaries within 30 words"
            },
            "Net Rating": {
                "type": "number",
                "description": "Overall Rating of the Product out of 5. use the 'Net Rating' of the input jsons to combine them logically"
            },
            "Justification": {
                "type": "string",
                "description": "Justification for the rating, within 20 words, summarised from the combined 'justifications' of all the input jsons"
            }
        }
  }
    }
]

In [ ]:
from openai import OpenAI
import os
import time
import ast
os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

def get_summaries_chat_gpt(product, prompt):


  num_reviews = len(reviews[product])
  summaries = []
  messages=[
    {"role": "system", "content": prompt},
  ]

  chunk_size = 15

  for i in range(0,num_reviews,chunk_size):

    client = OpenAI()
    messages=[
        {"role": "system", "content": prompt}]
    messages.append({"role":"user","content":f" {reviews[product][i:i+chunk_size]}"})
    completion = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=messages,
      functions = functions,
      function_call='auto'
    )

    print("Generated one summary")
    args = completion.choices[0].message.function_call.arguments
    try:
      output = json.loads(args)
      summaries.append(output)
    except:
      print ("Incorrect JSON format returned.")
    time.sleep(10)

  return summaries

def combine_summaries_chat_gpt(summaries, prompt, product):
  client = OpenAI()
  messages=[
    {"role": "user", "content": prompt},
  ]
  messages.append({"role":"user","content":f" {summaries}"})

  completion = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=messages
    )
  combined_summary = completion.choices[0].message.content

  messages=[
    {"role": "user", "content": PROMPT_TEXT_JSON_COMBINE},
  ]
  messages.append({"role":"user","content":f" {combined_summary}"})

  completion = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=messages,
      functions = functions_combine,
      function_call='auto'
    )

  args = completion.choices[0].message.function_call.arguments

  try:
    output = ast.literal_eval(args)
  except:
    output = json.loads(args)
  return {product:output}

### Summarize chunked reviews in JSON format

In [ ]:
%%time
product = "B00I11N2VO"
file_name = "B00I11N2VO_summaries_15.json"
summaries = get_summaries_chat_gpt(product,PROMPT_20)
with open(file_name, "w") as outfile:
    json.dump(summaries, outfile)

Generated one summary
Generated one summary
Generated one summary
CPU times: user 735 ms, sys: 46.2 ms, total: 781 ms
Wall time: 58.8 s


### Combine summaries into JSON format

In [ ]:
%%time
combine_file_name = "B00I11N2VO_combined_20.json"
combined_summary = combine_summaries_chat_gpt(summaries, PROMPT_COMBINE, product)
with open(combine_file_name, "w") as outfile:
    json.dump(combined_summary, outfile)

CPU times: user 290 ms, sys: 17.3 ms, total: 307 ms
Wall time: 33.4 s


### Generate tags and sentiments for tags for each review

In [ ]:
GET_TAGS = "You are an amazon review attribute generator. Use the function attributes to get 6 attributes. Given a set of 20 product reviews, you need to extract EXACTLY 6 attributes of the product from the reviews. The attribute is a SINGLE WORD characteristic that the product possesses that can be described as good or bad. The attribute does not denote what the product is, or what variant of the product it is. Give EXACTLY 6 attributes"

product = 'B00I11N2VO'
num_reviews = len(reviews[product])
messages=[
  {"role": "system", "content": GET_TAGS},
]
functions_attribute = [
    {
      "name": "attributes",
      "description": "Extract attributes from reviews",
      "parameters": {
        "type":"object",
        "properties":{
            "attributes": {
            "type": "array",
            "description":f"The list of 6 tags extracted from the reviews. Infer these from the review text.",
            "items":{
                "type":"string",
                "description":"a tag"
            }
          }
        }
      }
    }
]

for i in range(0,num_reviews,20):

  client = OpenAI()
  texts = [x['reviewText'] for x in reviews[product][i:i+20]]
  messages.append({"role":"user","content":f" Generate 6 tags for the attached reviews. reviews:{texts}"})

  completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=messages,
      functions= functions_attribute,
    function_call="auto",
    # temperature=1
  )

  print("Generated one attributes")
  args = completion.choices[0].message.function_call.arguments

Generated one attributes
Generated one attributes
Generated one attributes


In [ ]:
print(combine_summaries_chat_gpt(summaries,PROMPT_COMBINE_RATING))

Rating : 1
No reviews with a rating of 1 were found.

Rating : 2
No reviews with a rating of 2 were found.

Rating : 3
No reviews with a rating of 3 were found.

Rating : 4
One reviewer mentioned that the Velcro backing of the unit is not strong enough to hold such a heavy unit reliably. However, this reviewer was impressed with the solid construction and innovative features of the pedal. The algorithms used for the reverb are excellent, and the pedal covers a wide range of reverb styles. This pedal is a great choice for those on a budget who want a versatile and well-made reverb unit. There were two reviews with a rating of 4. One review mentioned that the pedal has some booming reverbs, but some of the modulation settings are wacky and over-the-top. The other review mentioned that the pedal does better spring reverb than the Boss' 63 Fender Reverb pedal.

Rating : 5
Many reviewers praised the pedal for its sound quality and versatility. They mentioned that it produces beautiful, ambi